In [44]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib.request as ur

In [49]:
def fetch_company_name_cik_table():
    CIKs = []
    companyNames = []
    path = '.'
    files = ['cik-list.txt']
    for f in files:

          with open (f, "r") as myfile:
            for line in myfile:
                #print(line)
                values=line.split(':')
                companyNames.append(values[len(values)-3])
                CIKs.append(values[(len(values)-2)].strip('0'))
    df = pd.DataFrame({'CIK': CIKs, 'company': companyNames})
    df.to_csv('CIK-mapping.csv')
    return df




In [4]:
#!/usr/bin/env python       

class GetData:
    
    def __init__(self):
        """
        Retrieves and stores the urllib.urlopen object for a given url
        """
        
    def generate_url(self,year):
        
        #generate the url for fetching the log files for every month's first day
        number_of_months=1
        while number_of_months < 13:
            if(number_of_months <10):
                url="http://www.sec.gov/dera/data/PublicEDGAR-log-file-data/"+year+"/Qtr1/log"+year+'%02d' % number_of_months+"01.zip"
            else:
                url="http://www.sec.gov/dera/data/PublicEDGAR-log-file-data/"+year+"/Qtr1/log"+year+str(number_of_months)+"01.zip"
            number_of_months=number_of_months+1
        #temp_url=download_data("http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr1/log20160101.zip")
        return self.download_data("http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2003/Qtr1/log20030301.zip")
        
    def download_data(self,url):

        

        #fetching the zip file name from the URL
        file_name=url.split("/")

        #Downloading data if not already present in the cache
        if(os.path.exists("Part_2_log_datasets/"+file_name[8])):
            print("Already present")

        else:
            urllib.request.urlretrieve(url, "Part_2_log_datasets/"+file_name[8])
            print("Download complete")

        #unzip the file and fetch the csv file
        zf = zipfile.ZipFile("Part_2_log_datasets/"+file_name[8]) 
        csv_file_name=file_name[8].replace("zip", "csv")
        zf_file=zf.open(csv_file_name)

        #create a dataframe from the csv
        df = pd.read_csv(zf_file)
        return df
        
#fetch the year for which the user wants logs
year = input('Enter the year for which you need to fetch the log files: ')
#calling the function to generate dynamic URL
get_data_obj=GetData()
df=get_data_obj.generate_url(year)
        


Enter the year for which you need to fetch the log files: 2003
Download complete


In [60]:
#convert all the integer column in int format
df['cik'] = df['cik'].astype('int')
df['zone'] = df['zone'].astype('int')
df['code'] = df['code'].astype('int')
#df['size'] = df['size'].astype('int')

print(df.head(25))

                 ip        date      time  zone      cik  \
0    129.110.39.jca  2003-03-01  00:00:00   500    97349   
1     61.115.76.jbf  2003-03-01  00:00:00   500   766351   
2     61.115.76.jbf  2003-03-01  00:00:01   500   902584   
3     61.115.76.jbf  2003-03-01  00:00:03   500   778207   
4    129.110.39.jca  2003-03-01  00:00:07   500    97349   
5     208.62.55.eib  2003-03-01  00:00:07   500    56824   
6    129.110.39.jca  2003-03-01  00:00:10   500    97349   
7     208.62.55.eib  2003-03-01  00:00:12   500    56824   
8    129.110.39.jca  2003-03-01  00:00:13   500    97349   
9   148.139.130.hhi  2003-03-01  00:00:16   500  1108205   
10   129.110.39.jca  2003-03-01  00:00:17   500    97349   
11   129.110.39.jca  2003-03-01  00:00:19   500    97349   
12    61.115.76.jbf  2003-03-01  00:00:21   500   778207   
13    67.81.137.eci  2003-03-01  00:00:21   500   857323   
14   203.200.34.ejd  2003-03-01  00:00:31   500  1125051   
15   129.110.39.jca  2003-03-01  00:00:3

In [61]:
#replacing empty strings with NaN 
df.replace(r'\s+', np.nan, regex=True)

,ip,date,time,zone,cik,accession,CIK_Accession_Anamoly_Flag,extention,code,size,idx,norefer,noagent,find,crawler,browser
0,129.110.39.jca,2003-03-01,00:00:00,500,97349,0000097349-01-000006,N,-0002.txt,200,3726.0,0.0,0.0,0.0,9.0,0.0,win
1,61.115.76.jbf,2003-03-01,00:00:00,500,766351,0000950134-03-003149,Y,.txt,200,995957.0,0.0,1.0,0.0,0.0,0.0,NaN
2,61.115.76.jbf,2003-03-01,00:00:01,500,902584,0000902584-03-000044,N,.txt,200,15520.0,0.0,1.0,0.0,0.0,0.0,NaN
3,61.115.76.jbf,2003-03-01,00:00:03,500,778207,9999999997-03-006003,Y,.txt,200,1670.0,0.0,1.0,0.0,0.0,0.0,NaN
4,129.110.39.jca,2003-03-01,00:00:07,500,97349,0000097349-01-000006,N,-index.htm,200,4331.0,1.0,0.0,0.0,1.0,0.0,win
5,208.62.55.eib,2003-03-01,00:00:07,500,56824,0000950124-03-000077,Y,-index.htm,200,2727.0,1.0,0.0,0.0,1.0,0.0,win
6,129.110.39.jca,2003-03-01,00:00:10,500,97349,0000097349-01-000006,N,-0003.txt,200,1211.0,0.0,0.0,0.0,9.0,0.0,win
7,208.62.55.eib,2003-03-01,00:00:12,500,56824,0000950124-03-000077,Y,k73883e10vqza.txt,200,158260.0,0.0,0.0,0.0,9.0,0.0,win
8,129.110.39.jca,2003-03-01,00:00:13,500,97349,0000097349-01-000006,N,-index.htm,200,4331.0,1.0,0.0,0.0,1.0,0.0,win
9,148.139.130.hhi,2003-03-01,00:00:16,500,1108205,0000927016-02-005853,Y,-index.htm,304,NaN,1.0,0.0,0.0,1.0,0.0,win


In [62]:

#replace all ip column NaN value by a default ip address 
df["ip"].fillna("255.255.255.255", inplace=True)

#replace all broser column NaN values by a string
df["browser"].fillna("Not Applicable", inplace=True)


#perform forward fill to replace NaN values by fetching the next valid value
df["date"].fillna(method='ffill')

#perform backward fill to replace NaN values by backpropagating and fetching the previous valid value
df["time"].fillna(method='bfill')

0        00:00:00
1        00:00:00
2        00:00:01
3        00:00:03
4        00:00:07
5        00:00:07
6        00:00:10
7        00:00:12
8        00:00:13
9        00:00:16
10       00:00:17
11       00:00:19
12       00:00:21
13       00:00:21
14       00:00:31
15       00:00:32
16       00:00:33
17       00:00:34
18       00:00:43
19       00:00:44
20       00:01:01
21       00:01:07
22       00:01:13
23       00:01:25
24       00:01:27
25       00:01:28
26       00:01:29
27       00:01:31
28       00:01:32
29       00:01:34
           ...   
43428    23:58:19
43429    23:58:20
43430    23:58:21
43431    23:58:23
43432    23:58:27
43433    23:58:32
43434    23:59:05
43435    23:59:06
43436    23:59:07
43437    23:59:08
43438    23:59:11
43439    23:59:12
43440    23:59:13
43441    23:59:14
43442    23:59:14
43443    23:59:16
43444    23:59:16
43445    23:59:17
43446    23:59:18
43447    23:59:20
43448    23:59:22
43449    23:59:39
43450    23:59:40
43451    23:59:41
43452    2

In [63]:
df['cik'] = df['cik'].astype('int')

In [64]:
df.insert(6, "CIK_Accession_Anamoly_Flag", "N")

ValueError: cannot insert CIK_Accession_Anamoly_Flag, already exists

In [ ]:
import pandas as pd
import numpy as np

#check if CIK and Accession number match. The Accession number is divided into three parts, CIK-Year-Number_of_filings_listed.
#the first part i.e the CIK must match with the CIK column. If not, there exists an anomaly

count=0;
for i in df['accession']:
    #fetch the CIK number from the accession number and convert it into integer
    list_of_fetched_cik_from_accession=[(int(i.split("-")[0]))]
    
    #check if the CIK number from the column and CIK number fetched from the accession number are equal
    if(df['cik'][count]!=list_of_fetched_cik_from_accession):
        df['CIK_Accession_Anamoly_Flag'][count]="Y"
        
    count=count+1
print(df.head(10))

   
    


C:\Users\Vasanti\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [58]:
#fetch the CIK_CompanyName concatenated dataframe
cik_companyname_dataframe=fetch_company_name_cik_table()

result = df.append(cik_companyname_dataframe)

print(result.head(10))

   CIK CIK_Accession_Anamoly_Flag             accession         browser  \
0  NaN                          N  0000097349-01-000006             win   
1  NaN                          Y  0000950134-03-003149  Not Applicable   
2  NaN                          N  0000902584-03-000044  Not Applicable   
3  NaN                          Y  9999999997-03-006003  Not Applicable   
4  NaN                          N  0000097349-01-000006             win   
5  NaN                          Y  0000950124-03-000077             win   
6  NaN                          N  0000097349-01-000006             win   
7  NaN                          Y  0000950124-03-000077             win   
8  NaN                          N  0000097349-01-000006             win   
9  NaN                          Y  0000927016-02-005853             win   

         cik   code company  crawler        date          extention  find  \
0    97349.0  200.0     NaN      0.0  2003-03-01          -0002.txt   9.0   
1   766351.0  200.0 